In [1]:
import cv2
import mediapipe as mp
import numpy as np
import copy
import itertools

In [2]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []
    
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        
        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))

    
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

def get_key(val):
    for key, value in labels_dict.items():
         if val == value:
             return key

    return "key doesn't exist"

In [3]:
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

labels_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,'Z':25,'space':26,'del':27,'nothing':28}


In [ ]:
#function will only work when trying to access from inside AWS notebook

import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.serializers import CSVSerializer
import json


endpoint = '' #get endpoint name from SageMaker > endpoints

predictor=sagemaker.predictor.Predictor(
    endpoint_name=endpoint,
    sagemaker_session=sagemaker.Session(),
    serializer=CSVSerializer())

while True:
    #success, image = cap.read()
    image = cv2.imread("W_test.jpg", cv2.IMREAD_COLOR)
    imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    predictions=""
    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks,results.multi_handedness):
            landmark_list = calc_landmark_list(imgRGB, hand_landmarks)
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            data = np.array([pre_processed_landmark_list], dtype=np.float32)
            predictions = predictor.predict(data)
            my_json = predictions.decode("utf-8").replace("'", '"')
            jsonData = json.loads(my_json)
            classes_x=np.argmax(np.squeeze(jsonData["predictions"]))
            print("\nLabel")
            print(get_key(classes_x))
            print("\n")
            cv2.putText(image, get_key(classes_x),(10,60), cv2.FONT_HERSHEY_PLAIN,3, (255,0,255),4)

    cv2.imshow("Results", image)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break


In [4]:
#Working USING LAMDA FUNCTION
import os
import io
import boto3
import json
import csv

# grab environment variables
ENDPOINT_NAME = ''
runtime= boto3.client('runtime.sagemaker', 
                     region_name='us-east-1', # make sure to set correct region
                     aws_access_key_id='',
                      aws_secret_access_key='')
while True:
    #success, image = cap.read()
    image = cv2.imread("W_test.jpg", cv2.IMREAD_COLOR)
    imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    predictions=""
    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks,results.multi_handedness):
            landmark_list = calc_landmark_list(imgRGB, hand_landmarks)
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            payload = ','.join(str(e) for e in pre_processed_landmark_list)
            response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                   ContentType='text/csv',
                                   Body=payload)
            
            jsonData = json.loads(response['Body'].read().decode())
            classes_x=np.argmax(np.squeeze(jsonData["predictions"]))
            print("\nLabel")
            print(get_key(classes_x))
            print("\n")
            cv2.putText(image, get_key(classes_x),(10,60), cv2.FONT_HERSHEY_PLAIN,3, (255,0,255),4)
    #remove the below break statement when you are ready to make mutiple api calls 
    break
    cv2.imshow("Results", image)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break




Label
W


